In [19]:
# Leer xml_id_fecha.json 

import json

with open("xml_id_fecha.json", "r") as archivo:
    datos = json.load(archivo)

In [20]:
# Third element in datos
print(datos[1])

{'file': 'sumario-20121002.xml', 'date': '02/10/2012', 'items': [['BOE-A-2012-12290', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12290.pdf'], ['BOE-A-2012-12291', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12291.pdf'], ['BOE-A-2012-12292', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12292.pdf'], ['BOE-A-2012-12293', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12293.pdf'], ['BOE-A-2012-12294', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12294.pdf'], ['BOE-A-2012-12295', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12295.pdf'], ['BOE-A-2012-12296', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12296.pdf'], ['BOE-A-2012-12297', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12297.pdf'], ['BOE-A-2012-12298', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12298.pdf'], ['BOE-A-2012-12299', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12299.pdf'], ['BOE-A-2012-12300', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12300.pdf'], ['BOE-A-2012-12303', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12303.pdf'], ['BOE-A-2012-12304', '/boe/dias/2012/10/02/pdfs/BOE-A-2012-12304.pdf'],

In [41]:
import datetime

# Convertir cadenas a objetos datetime
fecha_inicial = datetime.datetime.strptime("01/01/2023", "%d/%m/%Y")
fecha_final = datetime.datetime.strptime("31/12/2023", "%d/%m/%Y")

xml_id_rango = []
for xml in datos:
    try:
        fecha = datetime.datetime.strptime(xml["date"], "%d/%m/%Y")
    
        if fecha_inicial <= fecha <= fecha_final:
            xml_id_rango.append(xml)
    except:
        pass # Hay xmls con "fecha no encontrada" en vez de una fecha válida


In [42]:
print("First element: ", xml_id_rango[0])

First element:  {'file': 'sumario-20230809.xml', 'date': '09/08/2023', 'items': [['BOE-A-2023-18210', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18210.pdf'], ['BOE-A-2023-18211', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18211.pdf'], ['BOE-A-2023-18212', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18212.pdf'], ['BOE-A-2023-18213', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18213.pdf'], ['BOE-A-2023-18214', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18214.pdf'], ['BOE-A-2023-18215', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18215.pdf'], ['BOE-A-2023-18216', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18216.pdf'], ['BOE-A-2023-18217', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18217.pdf'], ['BOE-A-2023-18218', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18218.pdf'], ['BOE-A-2023-18219', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18219.pdf'], ['BOE-A-2023-18220', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18220.pdf'], ['BOE-A-2023-18221', '/boe/dias/2023/08/09/pdfs/BOE-A-2023-18221.pdf'], ['BOE-A-2023-18222', '/boe/dias/2023/08/09/pdfs/BOE-A-2

In [43]:
# Find first and last date in xml_id_rango
first_date = None
last_date = None
for xml in xml_id_rango:
    try:
        fecha = datetime.datetime.strptime(xml["date"], "%d/%m/%Y")
        if first_date is None or fecha < first_date:
            first_date = fecha
        if last_date is None or fecha > last_date:
            last_date = fecha
    except:
        pass

print("First date:", first_date)
print("Last date:", last_date)

First date: 2023-01-02 00:00:00
Last date: 2023-12-30 00:00:00


In [44]:
import os

# Lista de nombres de archivos PDF ya descargados sin la extensión
downloaded = [pdf.split('.')[0] for pdf in os.listdir('pdfs')]

to_download = []
for xml in xml_id_rango:
    ids_urls = xml["items"]
    # Obtener directamente los IDs desde ids_urls sin usar lambda
    for id_url in ids_urls:
        if id_url[0] not in downloaded:
            to_download.append(id_url)

In [45]:
print(f"To download: {to_download[0]}" )

To download: ['BOE-A-2023-8814', '/boe/dias/2023/04/10/pdfs/BOE-A-2023-8814.pdf']


In [46]:
print(f"Total to download: {len(to_download)}")

Total to download: 65241


In [47]:
import requests
import logging
base_url = "https://boe.es"  # Correct domain name
# Download the pdfs
for id_url in to_download:
    url = id_url[1]
    full_url = base_url + url
    print(f"Downloading {full_url}")
    filename = full_url.split("/")[-1]
    path = f"pdfs_range/{filename}"
    try:
        response = requests.get(full_url)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                f.write(response.content)
            logging.info(f"Downloaded PDF {filename} successfully.")
        else:
            logging.warning(f"Failed to download {filename}: Status {response.status_code}")
    except requests.RequestException as e:
        logging.error(f"Error downloading {filename}: {e}")

# En 20 mins he descargado 3010 pdfs.

# Solo para el ano 2023, hay 65241 pdfs para descargar. Esto es un locuron.

KeyboardInterrupt: 

In [48]:
import os
print(f"Total downloaded: {len(os.listdir('pdfs_range'))}")

Total downloaded: 3010


In [ ]:
# Read pdfs and filter them by the string "A. Contratación del Sector Público"

import os
import re
import PyPDF2

def read_pdf(file_path):
    with open(file_path, 'rb') as f:
        pdf = PyPDF2.PdfFileReader(f)
        text = ''
        for i in range(pdf.getNumPages()):
            text += pdf.getPage(i).extract_text()
    return text

def filter_text(text, pattern):
    return re.findall(pattern, text)

def filter_pdfs_by_pattern(pattern, folder = 'pdfs'):
    filtered_files = []
    for file in os.listdir(folder):
        if file.endswith('.pdf'):
            text = read_pdf(f'resoluciones/{file}')
            matches = filter_text(text, pattern)
            if matches:
                print(f'{file}: {matches}')
                filtered_files.append(file)
    return filtered_files

In [ ]:
# Filter by "A. Contratación del Sector Público"
filtered_files = filter_pdfs_by_pattern(pattern=r'A\. Contratación del Sector Público')

In [ ]:
# Create folder to store the filtered files
if not os.path.exists('licitaciones_filtered_pdfs/licitaciones'):
    os.makedirs('licitaciones_filtered_pdfs/licitaciones')

In [ ]:
# Filter by "Formalización del contrato"
contratacion = filter_pdfs_by_pattern(pattern=r'Formalización del contrato', folder='licitaciones_filtered_pdfs/licitaciones')

In [ ]:
# Create folder to store the filtered files
if not os.path.exists('licitaciones_filtered_pdfs/contratacion'):
    os.makedirs('licitaciones_filtered_pdfs/contratacion')